In [ ]:
import os
from pprint import pprint
from langchain.vectorstores.chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

def print_documents_metadata(documents):
    for i, document in enumerate(documents):
        print(f"Document {i}")
        pprint(document.metadata)

In [ ]:
persist_directory = os.environ.get("PERSIST_DIRECTORY", "/tmp")
embedding = OpenAIEmbeddings()

vectordb = Chroma(
    collection_name="hummingbot_documentation",
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [ ]:
vectordb._collection.count()

In [ ]:
# Comparing similarity search with max marginal relevance search
query = """Can you explain me how to configure a pure market making strategy?"""

# Similarity Search

In [ ]:
similar_documents = vectordb.similarity_search(query=query, k=10)
print_documents_metadata(similar_documents)

# Max Marginal Relevance Search

In [ ]:
mmr_documents = vectordb.max_marginal_relevance_search(query=query, k=5, fetch_k=10)
for i, document in enumerate(mmr_documents):
    print(f"Document {i}")
    pprint(document.page_content)

# SelfQueryRetriever

Using Runnables to construct the query

In [ ]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms.openai import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever


document_content_description = "Documentation of Hummingbot"
metadata_field_info = [
    AttributeInfo(
        name="url",
        description="The url where the information is collected",
        type="string",
    ),
    AttributeInfo(
        name="titles",
        description="The titles that are present in the page, all of them start with the symbol #. This field is important to understand the major topics that the documentation page includes.",
        type="string",
    ),
]

In [ ]:
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    enable_limit=True
)

In [ ]:
print_documents_metadata(retriever.get_relevant_documents(query, k=6))

In [ ]:
document_contents = "Hummingbot scripts"
llm = OpenAI(temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectordb,
    document_contents=document_contents,
    metadata_field_info=metadata_field_info,
    verbose=True
)

In [ ]:
relevant_docs = retriever.get_relevant_documents(query=query, k=2)

In [ ]:
print_documents_metadata(relevant_docs)

# Compressor

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.llms.openai import OpenAI

In [ ]:
# Wrap our vectorstore
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type="mmr"),
    verbose=True
)

In [ ]:
compressed_docs = compression_retriever.get_relevant_documents(query=query, k=6, fetch_k=15)

In [ ]:
compressed_docs[3].page_content

In [ ]:
len(compressed_docs)